# Neural Network Project
### Team Members: Ilias Sarantopoulos, Artem Tsikiridis
#### This project was done as part of the Machine Learning course for the Msc in Computer Science, Athens University of Economics and Business

$$ \frac{\partial E(w)}{\partial w_ij} = \delta_j x_i$$

We have the following 3 activation functions
1. $h(a)$= $\log(1+e^{a})$

2. $h(a)$ = $\frac{e^{a}-e^{-a}}{e^{a}+e^{-a}}$

3. $h(a)$= $\cos(a)$


In [33]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from PIL import Image


def load_data(file):
    list = []
    with open(file, 'r') as fp:
        for line in fp:
            list.append(line.strip("\n"))
    return list

if __name__ == '__main__':
    train_files = ['data/train%d.txt'% (i,) for i in range(10)]
    test_files = ['data/test%d.txt'% (i,) for i in range(10) ]
    counter = 0
    """for i in test_files:
        with open(i, 'r') as fp:
            counter += len(fp.readlines())
    print "train_data = ", counter
    b = np.array(zeros[0].split(" ")).reshape(28,28)
    img = Image.fromarray(b, 'RGB')
    imgplot = plt.imshow(img)
    tmp = []
    for i in train_files:
        with open(i, 'r') as fp:
            tmp += fp.readlines()
    print len(tmp)
    train_data = np.array([[j for j in i.split(" ")] for i in tmp])
    tmp = []
    for i in test_files:
        with open(i, 'r') as fp:
            tmp += fp.readlines()
    print len(tmp)
    test_data = np.array([[j for j in i.split(" ")] for i in tmp])
    print train_data.shape, test_data.shape"""
    truth = []
    test = []
    for i, _file in enumerate(train_files):
        count=0
        #print i, _file
        with open(_file , 'r') as fp:
            for line in fp:
                truth.append([1 if j == i else 0 for j in range(0,10)])
            # tried this way but didnt work
            #truth.append([1 if j == i else 0 for j in range(0,10)] for line in fp)
    ground_truth = np.array(truth)
    #print ground_truth.shape

(60000, 10)
